# Testing our YOLO checkpoints

In [10]:
from ultralytics import YOLO

model = YOLO("/data22/stu235269/TinyML-MT/model_checkpoints/yolo_classifier1.pt")

In [11]:
from pathlib import Path

image_folder = Path("/data22/stu235269/TinyML-MT/huggingface/inference")
selected_paths = list(image_folder.glob("*.jpg")) + list(image_folder.glob("*.png"))

In [12]:
preds = model.predict(
    selected_paths,
    imgsz=512,
    save=False,   # don't auto-save, we'll do it manually
    stream=False, # get a list of results
    verbose=True,  # optional: see output logs
    visualize=False
)


0: 512x512 (no detections), 1.4ms
1: 512x512 1 avocado, 1.4ms
2: 512x512 2 tomato sauces, 1.4ms
3: 512x512 1 cucumber, 4 tomato sauces, 1.4ms
4: 512x512 1 avocado, 2 fruit teas, 2 tomato sauces, 1.4ms
5: 512x512 2 fruit teas, 1 tomato sauce, 1.4ms
6: 512x512 3 avocados, 2 fruit teas, 2 tomato sauces, 1.4ms
7: 512x512 2 fruit teas, 1.4ms
8: 512x512 2 fruit teas, 1.4ms
9: 512x512 3 fruit teas, 1 tomato sauce, 1.4ms
10: 512x512 1 avocado, 1.4ms
11: 512x512 1 apple, 1 cucumber, 1.4ms
12: 512x512 1 apple, 1 lemon, 1.4ms
13: 512x512 1 apple, 1 lemon, 1.4ms
14: 512x512 (no detections), 1.4ms
15: 512x512 1 avocado, 1 fruit tea, 1.4ms
16: 512x512 1 tomato sauce, 1.4ms
17: 512x512 1 fruit tea, 2 tomato sauces, 1.4ms
18: 512x512 1 apple, 1 avocado, 1 cucumber, 4 fruit teas, 1 lemon, 1.4ms
19: 512x512 1 avocado, 1 banana, 2 lemons, 1 tomato sauce, 1.4ms
20: 512x512 1 apple, 2 cucumbers, 1 tomato sauce, 1.4ms
21: 512x512 2 avocados, 1 fruit tea, 4 tomato sauces, 1.4ms
22: 512x512 1 lemon, 4 tomato

In [13]:
import cv2

output_dir = Path("predicted_images_yolo1")
output_dir.mkdir(exist_ok=True)

for i, result in enumerate(preds):
    # result.plot() returns a NumPy array ready for cv2.imwrite (BGR)
    img_bgr = result.plot()

    # Use the original file name or create a numbered name
    original_path = selected_paths[i]
    out_path = output_dir / f"{original_path.stem}_pred.jpg"

    cv2.imwrite(str(out_path), img_bgr)
    print(f"Saved: {out_path}")

Saved: predicted_images_yolo1/IMG_8693_pred.jpg
Saved: predicted_images_yolo1/IMG_8694_pred.jpg
Saved: predicted_images_yolo1/IMG_8695_pred.jpg
Saved: predicted_images_yolo1/IMG_8696_pred.jpg
Saved: predicted_images_yolo1/IMG_8697_pred.jpg
Saved: predicted_images_yolo1/IMG_8698_pred.jpg
Saved: predicted_images_yolo1/IMG_8699_pred.jpg
Saved: predicted_images_yolo1/IMG_8700_pred.jpg
Saved: predicted_images_yolo1/IMG_8701_pred.jpg
Saved: predicted_images_yolo1/IMG_8702_pred.jpg
Saved: predicted_images_yolo1/IMG_8703_pred.jpg
Saved: predicted_images_yolo1/IMG_8704_pred.jpg
Saved: predicted_images_yolo1/image_1230_pred.jpg
Saved: predicted_images_yolo1/image_1231_pred.jpg
Saved: predicted_images_yolo1/image_1651_pred.jpg
Saved: predicted_images_yolo1/image_1674_pred.jpg
Saved: predicted_images_yolo1/generated_image_17_pred.jpg
Saved: predicted_images_yolo1/generated_image_18_pred.jpg
Saved: predicted_images_yolo1/generated_image_19_pred.jpg
Saved: predicted_images_yolo1/generated_image_20_p

## Test the COREML File Conversion
The App is worse than the original checkpoints of images fro m the phone so we want to see if nms=True hurst the accuracy or if the App has a problem

In [9]:
import coremltools as ct
from pathlib import Path
import numpy as np
from PIL import Image

# Path to your model.mlpackage
model_path = "/data22/stu235269/TinyML-MT/model_checkpoints/1-shot.mlpackage"

# Load the MLModel
mlmodel = ct.models.MLModel(model_path)

# Inspect input names
print("Inputs:")
for input in mlmodel.input_description:
    print(" -", input)

# Load and preprocess your image
image = Image.open("/data22/stu235269/TinyML-MT/huggingface/inference/IMG_8700.jpg").convert("RGB")
image = image.resize((640, 640))   # or do letterbox here if needed

# Convert to numpy array
input_array = np.array(image).astype(np.float32) / 255.0

# Check input shape (some models expect NCHW, others NHWC)
print("Input shape:", input_array.shape)

# Depending on the model, you might need to transpose (HWC->CHW)
# e.g., input_array = np.transpose(input_array, (2,0,1))

# Get input name (e.g., "image")
input_name = list(mlmodel.input_description)[0]

# Run prediction
outputs = mlmodel.predict({input_name: input_array})

# Print outputs
print("Outputs:")
for k, v in outputs.items():
    print(k, type(v), getattr(v, "shape", None))

Inputs:
 - image
 - iouThreshold
 - confidenceThreshold
Input shape: (640, 640, 3)


Exception: Model prediction is only supported on macOS version 10.13 or later.